<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
    <h3>GraphRAG Index Generation</h3>
    The following is an overview of the workflow:
    <ul>
    <li>Uses Microsoft GraphRAG library to construct a GraphRAG index for the synthetically generated dataset:</li>
        <ul>
            <li>Generates reformatted code-to-summary pairs</li>
            <li>Uses openai/gpt-oss-20b at the chat model</li>
            <li>Uses intfloat/e5-mistral-7b-instruct as the embedding model</li>
        </ul>
    </li>
    <li>Stores index in LanceDB database backed by Minio bucket</li>
    </ul>
</div>

In [1]:
##############################################
# Imports
##############################################
from minio import Minio
import os
import lancedb
import traceback
from minio.error import S3Error
import subprocess
import tracemalloc
tracemalloc.start()
import nest_asyncio
nest_asyncio.apply()
import utils

In [2]:
##############################################
# Generate GraphRAG index and store in LanceDB
##############################################

def generate_graphrag_index(dataset_name: str,
                            graphrag_source_path: str,
                            jsonl_source_path: str) :

    """
    Splits the provided jsonl file into seprate json files, then
    generates a GraphRAG index from the json files.
    Args:
        dataset_name: The source dataset name
        graphrag_source_path: The source path used by the GraphRAG index configuration
        jsonl_source_path: The jsonl source file
    Returns:
        None
    """

    ##############################################
    # Imports
    ##############################################
    from minio import Minio
    import os
    import lancedb
    from datasets import load_dataset
    import traceback
    import subprocess
    import tracemalloc
    tracemalloc.start()
    import nest_asyncio
    nest_asyncio.apply()
    import utils

    try:

        graph_rag_config_path = f"{graphrag_source_path}/settings.yaml"
        
        os.makedirs(f"{graphrag_source_path}/input", exist_ok=True)

        os.makedirs(f"{graphrag_source_path}/output", exist_ok=True)
        
        os.makedirs(os.path.dirname(jsonl_source_path), exist_ok=True)
        
        print(f"Preprocessing dataset {dataset_name}...")
        
        updated_dataset = utils.postprocess_dataset(dataset_name,
                                                    jsonl_source_path)

        print(f"Splitting dataset {dataset_name} into json files...")
    
        utils.split_jsonl_into_json_files(jsonl_source_path, f"{graphrag_source_path}/input")

        print("Running index...")
    
        result = subprocess.run(["bash", "graphrag.sh", graphrag_source_path, graph_rag_config_path], capture_output=True, text=True, check=False)
            
        print(f"\nSubprocess output: {result.stdout}")
        
        if result.stderr:
            
            raise Exception(f"Error processing GraphRAG command: {result.stderr}")
        
    except Exception as e:
        
        print(f"Error processing GraphRAG DB: {e}")
        traceback.print_exc()

In [3]:
##############################################
# Upload data to LanceDB
##############################################
def upload_graphrag_index_to_lancedb(graphrag_source_path: str, lancedb_minio_bucket_name: str, lancedb_db_name: str):
    """
    Uploads the GraphRAG index from the provided source path to the specified minio bucket.
    (Requires a valid Minio configuration which has been preconfigured using environment variables.)
    Args:
        graphrag_source_path: The source path for the GraphRAG index files
        lancedb_minio_bucket_name: The backing Minio bucket for the LanceDB database.
        lancedb_db_name: The LanceDB database name.
    Returns:
        None
    """

    ##############################################
    # Imports
    ##############################################
    from minio import Minio
    import os
    import lancedb
    from datasets import load_dataset
    import nest_asyncio
    import pandas as pd
    nest_asyncio.apply()
    
    
    try:

        print(f"Uploading index from {graphrag_source_path}...")
        
        graphrag_index_source_path = f"{graphrag_source_path}"
        
        db = lancedb.connect(f"s3://{lancedb_minio_bucket_name}/{lancedb_db_name}",
                             
            storage_options={
                "endpoint_url": os.getenv("AWS_S3_ENDPOINT"),
                
                "aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID"),
                
                "aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY"),
                
                "s3_force_path_style": "true",
                
                "allow_http": "true",
            }
        )
    
        local_db = lancedb.connect(f"{graphrag_source_path}/lancedb")
    
        all_tables = local_db.table_names()
    
        # Migrate Global Search tables
        print("Migrating global search tables...")
        
        for table_name in all_tables:
    
            try:
    
                local_table = local_db.open_table(table_name)
        
                local_df = local_table.to_pandas()
        
                db.create_table(table_name, data=local_df)
        
                print(f"{local_table} migrated.")
        
            except Exception as e:
                
                print(f"Error processing GraphRAG migration to Minio: {e}") 
    
        # Migrate Local Search tables
        print("Migrating local search tables...")
        
        for file_path in os.listdir(f"{graphrag_index_source_path}"):
            
            if file_path.endswith(".parquet"):
    
                try:
            
                    full_path = os.path.join(graphrag_index_source_path, file_path)
        
                    local_df = pd.read_parquet(full_path)
    
                    table_name = file_path.split(".", 1)[0]
        
                    db.create_table(table_name, data=local_df)
        
                    print(f"{table_name} migrated.")
        
                except Exception as e:
                    
                    print(f"Error processing GraphRAG migration to Minio: {e}")    
            
        print("Migration complete.")
        
    except Exception as e:
        
        print(f"Error processing GraphRAG migration to Minio: {e}")

### Generate GraphDB Index and Store in LanceDB
Start the indexing pipeline!

In [4]:
##############################################
# ColdFusion Baseline Index
##############################################
# try:
#     graphrag_path = "baseline_graph_rag/source"
    
#     jsonl_source_path = "baseline_graph_rag/json/graphrag.jsonl"
    
#     baseline_dataset_name = "oaawofolu/emerson"
    
#     bucket_name = "data"
    
#     baseline_lancedb_db_name = "cfcode-baseline"
    
#     generate_graphrag_index(baseline_dataset_name,
#                             graphrag_path,
#                             jsonl_source_path)
    
#     upload_graphrag_index_to_lancedb(f"{graphrag_path}/output", 
#                                      bucket_name, 
#                                      baseline_lancedb_db_name)
# except Exception as e:
        
#         print(f"Error processing ColdFusion Baselinex Index: {e}")

#         traceback.print_exc()

In [5]:
##############################################
# ColdFusion Sample Codebase Index
##############################################
# try:
#     graphrag_path = "graph_rag/source"
    
#     jsonl_source_path = "graph_rag/json/graphrag.jsonl"
    
#     prompt_dataset_name = "oaawofolu/cfcode-golfap"
    
#     bucket_name = "data"
    
#     prompt_lancedb_db_name = "cfcode-golfap-idx"
    
#     generate_graphrag_index(prompt_dataset_name,
#                             graphrag_path,
#                             jsonl_source_path)
    
#     upload_graphrag_index_to_lancedb(f"{graphrag_path}/output", 
#                                      bucket_name, 
#                                      prompt_lancedb_db_name)

# except Exception as e:
        
#         print(f"Error processing ColdFusion Sample Codebase Index: {e}")

#         traceback.print_exc()

### FULL PIPELINE
Execute full pipeline! 

In [6]:
##############################################
# Full Pipeline
##############################################
def graphrag_pipeline(git_repo: str):
    """
    Executes the full pipeline which generates a GraphRAG index from the given repo 
    and stores it in remote and local instances of LanceDB.
    """
    
    try:
        app_name = utils.get_unique_app_name_for_repo(git_repo) 
        
        if not app_name:
            
            raise Exception(f"""No unique app_name was found for git repo {git_repo} (ensure that the repo was registered during the SDG process)""")
        
        graphrag_path = f"graph_rag_{app_name}/source"
        
        jsonl_source_path = f"graph_rag_{app_name}/json/graphrag.jsonl"
        
        prompt_dataset_name = f"oaawofolu/cfcode-{app_name}"
        
        bucket_name = "data"
        
        prompt_lancedb_db_name = f"cfcode-{app_name}-idx"
    
        local_lancedb_path = f"local-lancedb-{app_name}"
        
        generate_graphrag_index(prompt_dataset_name,
                                graphrag_path,
                                jsonl_source_path)
        
        upload_graphrag_index_to_lancedb(f"{graphrag_path}/output", 
                                         bucket_name, 
                                         prompt_lancedb_db_name)
    
        utils.download_lancedb_index(bucket_name, prompt_lancedb_db_name, local_lancedb_path)
    
    except Exception as e:
            
            print(f"Error processing ColdFusion Sample Codebase Index: {e}")
    
            traceback.print_exc()

def query_index(git_repo: str,
                system_prompt: str,
                user_prompt: str,
                metadata: dict = {},
                cache_results: bool = False):
    """
    Queries the GraphRAG index associated with this git repo.
    If cache_results=True, the results will be cached in the backing LanceDB instance
    (pre-existing cache results will be overwritten).
    TODO: Support caching for multiple queries for each repo.
    """
    try:
        app_name = utils.get_unique_app_name_for_repo(git_repo) 
        
        if not app_name:
            
            raise Exception(f"""No unique app_name was found for git repo {git_repo} (ensure that the repo was registered during the SDG process)""")
    
        local_lancedb_path = f"local-lancedb-{app_name}"
        
        bucket_name = "data"

        _system_prompt, _user_prompt = system_prompt, user_prompt

        for key in metadata:

            _system_prompt = _system_prompt.replace("{{" + key + "}}", metadata[key])

            _user_prompt = _user_prompt.replace("{{" + key + "}}", metadata[key])
        
        output = utils.query_lancedb_graphrag_index(root=local_lancedb_path, 
                                           config=f"settings.yaml", 
                                           prompt=f"{_system_prompt}\n{_user_prompt}", 
                                           response_type="text format",
                                           method="global")

        if cache_results:
            
            utils.create_or_update_indexing_job(git_repo, bucket_name, results=output)

        return output
    
    except Exception as e:
            
            print(f"Error querying GraphRAG index: {e}")
    
            traceback.print_exc()
    


### GraphRAG Queries
Perform queries against the GraphRAG index

In [7]:
##############################################
# Full Pipeline
##############################################
input1 = """
This code represents an application called Golfap, which is 
a social golf scorekeeping app that makes it easy to track scores, bets, and games with friends.
"""

input2 = """
This code represents a ColdFusion based Content Management System.
"""

input3 = """
This code creates a Calendar widget using ColdFusion.
"""

input4 = """
This code represents an app called CFLipsum,
a simple component that uses the lipsum.com XML feed to generate placeholder Lorem Ipsum text.
"""

input5 = """
This code represemts the modification of the Fusebox 5.5.1 Implicit (no-XML) Skeleton application.
"""

prompt1 = """
Your task is to analyze this code and generate a software design document.
This document should include a concise explanation of the purpose of the code.
If the provided snippet does not appear to be valid code, indicate that this is not valid code.
Stop after you have finished writing the 4 sections described below.
Do not stray from the functionality provided in the codebase. 
Do not change the file paths or create new file paths. Display them exactly as they are shown.
Also do not create new functions or components that do not exist in the code. Stick strictly to the code provided.
Use a separate line for each component, function or feature. Do not combine multiple components into 1 line.

The directory structure of the code is as follows:
{{directory_structure}}

Your analysis must include the following sections:

**1. Summary:** 
* Provide a clear and concise explanation of the purpose of the code.

**2. Components:** 
* Provide a concise list of all the important ColdFusion relevant components you can find. 
* Include the following:
    * List of primary modules and their responsibilities: include the file paths if they are available
    * List of primary web pages and their responsibilitites: include the file paths if they are available
    * List of ColdFusion components and their responsibilities: include the file paths if they are available
    * List of reusable templates and their responsibilities: include the file paths if they are available
    * List of the names of custom components and their responsibilities (if any), exactly as shown in the code, such as cfcomponents, user-defined functions and any other custom components you can find; include the file paths if they are available
    
**3. Conceptual Domain:** 
* Generate a concise outline of the conceptual domain model associated with this code.
* Include the current state of the domain objects based on information extracted from the code.
* Include the purpose of each domain object extracted from the conceptual domain model.

**4. Physical Domain:** 
* Generate a concise outline of the physical domain model associated with this code.
* Include the current state of the physical domain objects based on information extracted from the code.
* Include the purpose of each object extracted from the physical domain model.

"""

prompt2 = """
Here is a summary of the codebase:
Generate a language-agnostic Software Design Document for this code. 
The document should be structured, professional, and suitable for both technical and non-technical stakeholders.
Also exclude any details that link the requirements to ColdFusion or any other specific programming language or framework.
Instead, it should focus on universal concepts and architecture that can be implemented in any programming language.
Stop after you have finished writing the 3 sections described below.

The SDD must include the following sections:

**1. System Architecture**
*   **Overall Design:** Describe the main architectural patterns used and how the different components interact. 
*   **Key Components:** List of the primary modules, classes, domain model and services and their responsibilities.

**2. Functional Requirements**
*   **Input Handling:** How does the system accept inputs?
*   **Data Processing:** List of the main logic, algorithms, and data transformations.
*   **Output Generation:** How are results produced and presented?

**3. Business Requirements**
*   **Business Rules:** List of specific rules that govern how the business operates, which the software must enforce.
*   **Success Criteria:** List of measurable criteria to determine if the project is successful, also known as acceptance criteria.

Only use the context provided in the summary above. Do not stray from the context provided in the summary. 
Include named components and objects based on the context where it makes sense.
"""

In [8]:
import os
git_repo_param = os.getenv("PIPELINE_PARAM_GIT_REPO", "https://github.com/holtonma/cf_golfap.git")
enabled_param = os.getenv("PIPELINE_PARAM_ENABLED", "true")
if enabled_param == "true":
    graphrag_pipeline(git_repo_param)
    query_index(git_repo_param,
            input1, 
            prompt1,
            metadata = {"directory_structure": utils.get_directory_structure(git_repo_param)},
            cache_results = True)

# graphrag_pipeline("https://github.com/kishore31/CheckMate-CMS") 
# query_index("https://github.com/holtonma/cf_golfap.git", 
#             input2, 
#             prompt1,
#             metadata = {"directory_structure": utils.get_directory_structure("https://github.com/kishore31/CheckMate-CMS")},
#             cache_results = True)

# graphrag_pipeline("https://github.com/ehynds/cfml")
# query_index("https://github.com/ehynds/cfml", 
#             input3, 
#             prompt1,
#             metadata = {"directory_structure": utils.get_directory_structure("https://github.com/ehynds/cfml")},
#             cache_results = True)

# graphrag_pipeline("https://github.com/timblair/cflipsum")
# query_index("https://github.com/timblair/cflipsum", 
#             input4, 
#             prompt1,
#             metadata = {"directory_structure": utils.get_directory_structure("https://github.com/timblair/cflipsum")},
#             cache_results = True)

# graphrag_pipeline("https://github.com/illuminerdi/fusebox_implicit_skelly") 
# query_index("https://github.com/illuminerdi/fusebox_implicit_skelly", 
#             input5, 
#             prompt1,
#             metadata = {"directory_structure": utils.get_directory_structure("https://github.com/illuminerdi/fusebox_implicit_skelly")},
#             cache_results = True)

Preprocessing dataset oaawofolu/cfcode-golfap...


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Splitting dataset oaawofolu/cfcode-golfap into json files...
Successfully converted 'graph_rag_golfap/json/graphrag.jsonl' to json files under'graph_rag_golfap/source/input'.
Running index...

Subprocess output: Initializing GraphRAG index...
Copying settings.yaml...
Configuring prompts...
/opt/app-root/lib64/python3.11/site-packages/litellm/llms/custom_httpx/async_client_cleanup.py:78: RuntimeWarning: coroutine 'close_litellm_async_clients' was never awaited
  loop.close()
Populating GraphRAG index...
Starting pipeline with workflows: load_input_documents, create_base_text_units, create_final_documents, extract_graph, finalize_graph, extract_covariates, create_communities, create_final_text_units, create_community_reports, generate_text_embeddings
Starting workflow: load_input_documents

Workflow complete: load_input_documents
Starting workflow: create_base_text_units
  1 / 1 ............................................................................................
  1 / 1 .........

'**1. Summary**  \nGolfap is a social golf score‑keeping web application that lets users record scores, place bets, and play games with friends. It presents news feeds, tournament data, player statistics, and leaderboards through a set of ColdFusion pages and a JSON component that serialises data for client‑side consumption. The code is organized into a *news* module, a *pgatour* module, and a shared *tabs.cfm* navigation template.  \n[Data: Reports (5, 173, 153, 142, 60, +more)]\n\n---\n\n**2. Components**  \n\n*Primary modules*  \n- **/news** – handles news aggregation, user registration, score encoding, and leaderboard display.  \n- **/pgatour** – manages PGA Tour integration, tournament listings, player data, and tour‑specific news.  \n- **/tabs.cfm** – global tab navigation shared across modules.  \n\n*Primary web pages*  \n- **/news/index.cfm** – landing page for the news module.  \n- **/news/page.cfm** – generic page wrapper that includes header, scripts, and analytics.  \n- **/

In [ ]:
if enabled_param == "true":
    utils.fetch_indexing_job("https://github.com/holtonma/cf_golfap.git", "data")